In [1]:
from sympy.physics.quantum.spin import Rotation
from numpy import *
import numpy as np
import sympy
from sympy import I, sin, cos, exp, pi, symbols

In [2]:
def f_App_A2(l1,l2,m,ri,rj):
    f = (-1)**(m+l2) \
    *math.factorial(l1+l2) \
    *(math.factorial(l1+abs(m)) \
     *math.factorial(l1-abs(m)) \
     *math.factorial(l2+abs(m)) \
     *math.factorial(l2-abs(m)) )**(-.5) \
    *ri**l1 *rj**l2
    return f

In [3]:
def euler_to_matrix(omega):
    # 3-1-3 extrinsic ... see https://en.wikipedia.org/wiki/Rotation_formalisms_in_three_dimensions
    alpha = omega[0]
    beta = omega[1]
    gamma = omega[2]
    c1 = cos(alpha); c2 = cos(beta); c3 = cos(gamma)
    s1 = sin(alpha); s2 = sin(beta); s3 = sin(gamma)
    A11 = c1*c3-c2*s1*s3
    A12 = c1*s3-c2*c3*s1
    A13 = s1*s2
    A21 = c3*s1+c1*c2*s3
    A22 = c1*c2*c3-s1*s3
    A23 = -c1*s2
    A31 = s2*s3
    A32 = c3*s2
    A33 = c2
    Re = matrix([[A11, A12, A13],[A21, A22, A23],[A31, A32, A33]]).astype(float)
    return Re


In [4]:
def matrix_to_euler(Re):
    A31 = Re[2,0]
    A32 = Re[2,1]
    A33 = Re[2,2]
    A13 = Re[0,2]
    A23 = Re[1,2]
    beta = np.arccos(A33)
    gamma = np.arctan2(A31,A32)
    alpha = np.pi - np.arctan2(A13,A23)
    omega = [alpha,beta,gamma]
    return omega

omega = [1.9, 2.3, 0.1]
Re = euler_to_matrix(omega)
omeganew = matrix_to_euler(Re)
Renew = euler_to_matrix(omeganew)
print (omega)
print (omeganew)
print (Re)
print (Renew)

# Checking that we understand these rotations
unitx = matrix([1,0,0]).T
unity = matrix([0,1,0]).T
unitz = matrix([0,0,1]).T
omega = [0,.1,0]
unitz_rotated = euler_to_matrix(omega)*unitz; print ("beta rotates unit z to:\n", unitz_rotated)
omega = [.1,0,0]
unitz_rotated = euler_to_matrix(omega)*unitz; print ("alpha rotates unit z to:\n", unitz_rotated)


[1.9, 2.3, 0.1]
[1.8999999999999999, 2.2999999999999998, 0.10000000000000002]
[[-0.25872979  0.5950721   0.70566091]
 [ 0.96307666  0.11985161  0.24107872]
 [ 0.0744463   0.74197979 -0.66627602]]
[[-0.25872979  0.5950721   0.70566091]
 [ 0.96307666  0.11985161  0.24107872]
 [ 0.0744463   0.74197979 -0.66627602]]
beta rotates unit z to:
 [[ 0.        ]
 [-0.09983342]
 [ 0.99500417]]
alpha rotates unit z to:
 [[ 0.]
 [ 0.]
 [ 1.]]


In [5]:
def expand_in_R(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,R):
    brackets = 0
    for l1 in range(l1max+1):
        for l2 in range(l2max+1):
            lmin = min(l1,l2)
            for m in range (-lmin,lmin+1):
                D1i = Rotation.D(l1, m, 0,omega1i_bar[0], omega1i_bar[1], omega1i_bar[2]).doit()
                D2j = Rotation.D(l2, m, 0,omega2j_bar[0], omega2j_bar[1], omega2j_bar[2]).doit()
                f = f_App_A2(l1,l2,m,ri,rj)
                brackets += D1i*conj(D2j)*f/R**(l1+l2)
    rij = eval(str(R / brackets))
    try:
        rij = rij.expand(complex=True)
    except:
        pass
    rij = eval(str(rij))
    return rij, brackets

In [6]:
def expand_in_omega_R(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1,omega2,R):
    brackets = 0
    for l1 in range(l1max+1):
        for l2 in range(l2max+1):
            lmin = min(l1,l2)
            for m in range (-lmin,lmin+1):
                D1i = 0
                for mp in range(-l1,l1+1):
                    temp1 = Rotation.D(l1, m, mp, omega1[0], omega1[1], omega1[2]).doit()
                    temp2 = Rotation.D(l1, mp, 0, omega1i_bar[0], omega1i_bar[1], omega1i_bar[2]).doit()
                    D1i += temp1*temp2
                D2j = 0
                for mp in range(-l2,l2+1):
                    temp1 = Rotation.D(l2, m, mp, omega2[0], omega2[1], omega2[2]).doit()
                    temp2 = Rotation.D(l2, mp, 0, omega2j_bar[0], omega2j_bar[1], omega2j_bar[2]).doit()
                    D2j += temp1*temp2
                f = f_App_A2(l1,l2,m,ri,rj)
                brackets += D1i*conj(D2j)*f/R**(l1+l2)
    rij = eval(str(R / brackets))
    try:
        rij = rij.expand(complex=True)
        brackets = brackets.expand(complex=True)
    except:
        pass
    return rij, brackets

In [7]:
def expand_in_omega_R_psi(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1,omega2,R,psi):
    brackets = 0
    for l1 in range(l1max+1):
        for l2 in range(l2max+1):
            lmin = min(l1,l2)
            for m in range (-lmin,lmin+1):
                D1i = 0
                for mp in range(-l1,l1+1):
                    temp1 = 0
                    for mpp in range(-l1,l1+1):
                        tempp1 = Rotation.D(l1, m, mpp,  0,psi,0).doit()
                        tempp2 = Rotation.D(l1, mpp, mp, omega1[0], omega1[1], omega1[2]).doit()
                        temp1 += tempp1*tempp2
                    temp2 = Rotation.D(l1, mp, 0, omega1i_bar[0], omega1i_bar[1], omega1i_bar[2]).doit()
                    D1i += temp1*temp2
                D2j = 0
                for mp in range(-l2,l2+1):
                    temp1 = 0
                    for mpp in range(-l2,l2+1):
                        tempp1 = Rotation.D(l2, m, mpp,  0,psi,0).doit()
                        tempp2 = Rotation.D(l2, mpp, mp, omega2[0], omega2[1], omega2[2]).doit()
                        temp1 += tempp1*tempp2
                    temp2 = Rotation.D(l2, mp, 0, omega2j_bar[0], omega2j_bar[1], omega2j_bar[2]).doit()
                    D2j += temp1*temp2
                f = f_App_A2(l1,l2,m,ri,rj)
                brackets += D1i*conj(D2j)*f/R**(l1+l2)
    rij = eval(str(R / brackets))
    try:
        rij = rij.expand(complex=True)
        brackets = brackets.expand(complex=True)
    except:
        pass
    return rij, brackets

In [8]:
def evaluateRomegas(rij_sym, \
               R_sym, R_num, \
               omega1_sym, omega1_num, \
               omega2_sym, omega2_num \
             ):
    # Evaluate for numerical values of R, psi, etc
    alpha1_sym =  omega1_sym[0]; alpha1_num =  omega1_num[0]
    beta1_sym  =  omega1_sym[1]; beta1_num  =  omega1_num[1]
    gamma1_sym =  omega1_sym[2]; gamma1_num =  omega1_num[2]
    alpha2_sym =  omega2_sym[0]; alpha2_num =  omega2_num[0]
    beta2_sym  =  omega2_sym[1]; beta2_num  =  omega2_num[1]
    gamma2_sym =  omega2_sym[2]; gamma2_num =  omega2_num[2] 
    rij = rij_sym.subs([ \
                  (R_sym,R_num),\
                  (alpha1_sym,alpha1_num),\
                  (beta1_sym,beta1_num),\
                  (gamma1_sym,gamma1_num),\
                  (alpha2_sym,alpha2_num),\
                  (beta2_sym,beta2_num),\
                  (gamma2_sym,gamma1_num),\
               ]
               )
    rij = eval(str(rij))
    return rij

def evaluatepsi(rij_sym, \
               psi_sym, psi_num):

    rij = rij_sym.subs(psi_sym,psi_num)
    return rij

In [9]:
def showbrackets(brackets_sym):

    # This is to look at the expansion of the term in brackets
    alpha1, beta1, gamma1 = symbols('alpha1 beta1 gamma1', real=True)
    alpha2, beta2, gamma2 = symbols('alpha2 beta2 gamma2', real=True)
    R, psi = symbols('R psi', real=True)

    # These substitutions are just for aesthetics
    brackets_sym = brackets_sym.subs([ \
                  (R_sym,R),\
                  (alpha1_sym,alpha1),\
                  (beta1_sym,beta1),\
                  (gamma1_sym,gamma1),\
                  (alpha2_sym,alpha2),\
                  (beta2_sym,beta2),\
                  (gamma2_sym,gamma1),\
                 ])

    try:
        brackets_sym = brackets_sym.subs(psi_sym,psi)
    except:
        pass

    # Print the term in brackets
    #sympy.pretty_print(brackets_sym)

    # Print terms according to powers of 1/R
    x = symbols('x', real=True)
    test = brackets_sym.subs(R, 1/x)
    for i in range(4):
        term = test.diff(x,i).subs(x,0)/math.factorial(i)
        print ("Coefficient of 1/R^", i)
        #sympy.pretty_print(term)
        print(term)
        print ("\n")

In [10]:
# Specify the order of the expansion
l1max = 2
l2max = 2
ri = 1
rj = 1

# This is the distance between frames
R_num = 10

In [11]:
# This is the original formulation

# Specify the atomic configurations
omega1i_bar = [0.1,1.5,0]  
omega2j_bar = [0.1,0.5,0]

# Expand
print (omega1i_bar)
print (omega2j_bar)
rij, brackets = expand_in_R(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,R_num)
print (rij)

[0.1, 1.5, 0]
[0.1, 0.5, 0]
10.8160786565


In [12]:
# Here, start with atoms along the z-axis, and rotate the frames to match the above
# This should equal the cell above

# Specify the atomic configurations
omega1i_bar = [0,0,0]
omega2j_bar = [0,0,0]

# Specify a frame rotation for each atom
omega1i_num = [0.1,1.5,0]
omega2j_num = [0.1,0.5,0]

# Translate to rotation matrices, combine them, and get back the euler angles
R_omega1i_bar = euler_to_matrix(omega1i_bar)
R_omega2j_bar = euler_to_matrix(omega2j_bar)
R_omega1i = euler_to_matrix(omega1i_num)
R_omega2j = euler_to_matrix(omega2j_num)
R_omega1i_omega1i_bar = R_omega1i*R_omega1i_bar
R_omega2j_omega2j_bar = R_omega2j*R_omega2j_bar
omega1i_bar_p = matrix_to_euler(R_omega1i_omega1i_bar)
omega2j_bar_p = matrix_to_euler(R_omega2j_omega2j_bar)

# Expand using these new euler angles (which should be the same as the previous ones)
print (omega1i_bar_p)
print (omega2j_bar_p)
rij, brackets = expand_in_R(l1max,l2max,ri,rj,omega1i_bar_p,omega2j_bar_p,R_num)
print (rij)

[0.10000000000000009, 1.5, 0.0]
[0.10000000000000009, 0.49999999999999989, 0.0]
10.8160786565


In [13]:
# Now try an omega-R expansion of the Wigner D-matrices*omega-bar instead
# This should equal the cell above
rij, brackets = expand_in_omega_R(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1i_num,omega2j_num,R_num)
print (rij)

10.8160786565


In [14]:
# Now try an R-expansion of the Wigner D-matrices alone
# This should equal the cell above
rij, brackets = expand_in_R(l1max,l2max,ri,rj,omega1i_num,omega2j_num,R_num)
print (rij)

10.8160786565


In [15]:
# Next we'll try rotating each with a common omega1, omega2
# This will be different from the above because each molecule is rotated
omega1_num = [.1,.2,.3]
omega2_num = [.2,.3,.4]
rij, brackets = expand_in_omega_R(l1max,l2max,ri,rj,omega1i_num,omega2j_num,omega1_num,omega2_num,R_num)
print (rij)

10.8351078967783 - 7.02055563657329e-18*I


In [16]:
# # Now try an expansion of the Wigner D-matrices, but with symbolic variables

# # Create the symbolic values
# R_sym = symbols('R_sym', real=True)
# alpha1_sym, beta1_sym, gamma1_sym = symbols('alpha1_sym beta1_sym gamma1_sym', real=True)
# alpha2_sym, beta2_sym, gamma2_sym = symbols('alpha2_sym beta2_sym gamma2_sym', real=True)
# omega1_sym = [alpha1_sym, beta1_sym, gamma1_sym];
# omega2_sym = [alpha2_sym, beta2_sym, gamma2_sym]; 

# # Expand
# rij_sym, brackets_sym = expand_in_omega_R(l1max,l2max,ri,rj,omega1i_num,omega2j_num,omega1_sym,omega2_sym,R_sym)
# #print (rij_sym)

In [17]:
# # To check, evaluate the symbolic variables at numerical values
# # This should equal the cell above (but there's roundoff)
# rij_eval = evaluateRomegas(rij_sym, \
#                R_sym, R_num, \
#                omega1_sym, omega1_num, \
#                omega2_sym, omega2_num \
#              )
# print rij_eval

In [18]:
# # Also good to check the brackets
# showbrackets(brackets_sym)

In [19]:
# Now we try an expansion of the Wigner D-matrices, with angle psi applied to both
# This result should be the same as the cell above since psi=0

# This is the angle 
psi_num = 0

# Expand 
rij, brackets = \
    expand_in_omega_R_psi(l1max,l2max,ri,rj,omega1i_num,omega2j_num,omega1_num,omega2_num,R_num,psi_num)
print (rij)

10.8351078967783 - 7.02055563657329e-18*I


In [20]:
# # Now we try the same with symbolic variables

# # Create the symbolic variable
# psi_sym = symbols('psi_sym', real=True)

# # Expand
# rij_sym, brackets_sym = \
#     expand_in_omega_R_psi(l1max,l2max,ri,rj,omega1i_num,omega2j_num,omega1_sym,omega2_sym,R_sym,psi_sym)
# #print rij_sym

In [21]:
# # To check, evaluate the symbolic variables at numerical values 
# # This should equal the cell above (but there's roundoff)
# rij_eval1 = evaluateRomegas(rij_sym, \
#                R_sym, R_num, \
#                omega1_sym, omega1_num, \
#                omega2_sym, omega2_num \
#              )
# rij_eval2 = evaluatepsi(rij_eval1, psi_sym, psi_num)
# print rij_eval2

In [22]:
# # Also good to check the brackets
# showbrackets(brackets_sym)

In [23]:
# Now we try an expansion of the Wigner D-matrices, with angle psi applied to both
# This result will be different from the cell above (because psi != 0)

# This is the angle 
psi_num = .2

# Expand 
rij, brackets = \
    expand_in_omega_R_psi(l1max,l2max,ri,rj,omega1i_num,omega2j_num,omega1_num,omega2_num,R_num,psi_num)
print (rij)

10.8723489263439 + 6.24786528377259e-18*I


In [24]:
# # To check, evaluate the symbolic variables at numerical values 
# # This should equal the cell above with psi != 0 (but there's roundoff)
# rij_eval1 = evaluateRomegas(rij_sym, \
#                R_sym, R_num, \
#                omega1_sym, omega1_num, \
#                omega2_sym, omega2_num \
#              )
# rij_eval2 = evaluatepsi(rij_eval1, psi_sym, psi_num)
# print rij_eval2

In [26]:
# Testing against the geometrically obtained distances
R = matrix([0,0,R_num]).T

R_omega1i_num = euler_to_matrix(omega1i_num)
R_omega2j_num = euler_to_matrix(omega2j_num)
V1 = R_omega1i_num*matrix([0,0,ri]).T; print (V1)
V2 = R_omega2j_num*matrix([0,0,rj]).T; print (V2)
Rij = linalg.norm(V1-(V2+R)); print ("Actual distance case 1 is", Rij,"\n")

R_omega1_num = euler_to_matrix(omega1_num)
R_omega2_num = euler_to_matrix(omega2_num)
V11 = R_omega1_num*V1; print (V11)
V22 = R_omega2_num*V2; print (V22)
Rij = linalg.norm(V11-(V22+R)); print ("Actual distance case 2 is", Rij,"\n")

R_psi_num = euler_to_matrix([0,psi_num,0])
V111 = R_psi_num*V11; print (V111)
V222 = R_psi_num*V22; print (V222)
Rij = linalg.norm(V111-(V222+R)); print ("Actual distance case 3 is", Rij,"\n")

[[ 0.09958333]
 [-0.99251167]
 [ 0.0707372 ]]
[[ 0.04786269]
 [-0.47703041]
 [ 0.87758256]]
Actual distance case 1 is 10.8192561016 

[[-0.10588465]
 [-0.8711449 ]
 [-0.11320102]]
[[-0.00747825]
 [-0.60244171]
 [ 0.7140508 ]]
Actual distance case 2 is 10.8310326049 

[[-0.10588465]
 [-0.83129043]
 [-0.28401431]]
[[-0.00747825]
 [-0.73229298]
 [ 0.58013064]]
Actual distance case 3 is 10.8650416352 

